In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder, Rect3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO


include("../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Hand Visualization

In [5]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.00, -0.01, 0.31701]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.0
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.31701
 0.0  0.0  0.0   1.0

In [3]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "world")
#display_transform(ls, T)

box = Rect3f(Point3f(box_position) - Vec3f(box_dimensions), 2*Vec3f(box_dimensions))
mesh!(ls, box; color=:magenta, transparency=true)

# Display figure
display(fig)

LoadError: UndefVarError: `box_position` not defined

In [215]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))

Dict{String, VMRobotControl.CoordinateData} with 55 entries:
  "rh_lfdistal_mass_coord" => FramePoint{Float64, String}(rh_lfdistal, [0.0, 0.0, 0.012])
  "rh_rfdistal_mass_coord" => FramePoint{Float64, String}(rh_rfdistal, [0.0, 0.0, 0.012])
  "rh_thdistal_mass_coord" => FramePoint{Float64, String}(rh_thdistal, [0.0, 0.0, 0.01375])
  "rh_rftip_mass_coord" => FrameOrigin{String}(rh_rftip)
  "rh_palm_mass_coord" => FramePoint{Float64, String}(rh_palm, [0.0, 0.0, 0.035])
  "rh_rfknuckle_mass_coord" => FrameOrigin{String}(rh_rfknuckle)
  "rh_wrist_mass_coord" => FramePoint{Float64, String}(rh_wrist, [0.0, 0.0, 0.029])
  "rh_ffdistal_inertia_coord" => FrameAngularVelocity{String}("rh_ffdistal")
  "rh_thhub_mass_coord" => FrameOrigin{String}(rh_thhub)
  "rh_rfmiddle_mass_coord" => FramePoint{Float64, String}(rh_rfmiddle, [0.0, 0.0, 0.0125])
  "rh_mftip_mass_coord" => FrameOrigin{String}(rh_mftip)
  "rh_ffdistal_mass_coord" => FramePoint{Float64, String}(rh_ffdistal, [0.0, 0.0, 0.012])
  "rh_m

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [4]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Addition of the multiples springs/dampers

Virtual Mechanism system

In [5]:
vms = VirtualMechanismSystem("myShadowVMS", robot)
root = root_frame(vms.robot)

"world"

Repulsive Box

In [6]:
# box_position = SVector(0.0, -0.065, 0.32)
# box_dimensions = [0.03, 0.02, 0.03]

box_position = SVector(0.055, -0.01, 0.37)
box_dimensions = [0.01, 0.05, 0.05] 

add_coordinate!(vms,  ConstCoord(box_position);  id="box position")
add_coordinate!(vms, ConstCoord(box_dimensions[1]); id="box dimension 1")
add_coordinate!(vms, ConstCoord(box_dimensions[2]); id="box dimension 2")
add_coordinate!(vms, ConstCoord(box_dimensions[3]); id="box dimension 3")

# add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "box position") ; id = "fftip box diff" )
# for j in 1:3
#     add_coordinate!(vms, CoordSlice("fftip box diff", SVector(j)); id = "fftip box diff dimension $(j)")
#     add_coordinate!(vms, CoordNorm("fftip box diff dimension $(j)"); id = "fftip box norm dimension $(j)")
#     add_coordinate!(vms, CoordDifference("fftip box norm dimension $(j)","box dimension $(j)"); id = "shifted fftip box norm dimension $(j)")

#     add_component!(vms, ReLUSpring(0.0, "shifted fftip box norm dimension $(j)", true); id="fftip dimension $(j) repulsive spring")
#     add_component!(vms, RectifiedDamper(0.0, "fftip box norm dimension $(j)", (0.0, 1.1*box_dimensions[j]), true, false); id="fftip dimension $(j) damper")
# end

add_coordinate!(vms, FramePoint(".robot.rh_palm", SVector(0. , 0., 0.07)); id="second palm point")
repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_mftip_mass_coord", ".robot.rh_rftip_mass_coord",".robot.rh_lftip_mass_coord" , 
                    ".robot.rh_thtip_mass_coord", ".robot.rh_ffmiddle_mass_coord",".robot.rh_mfmiddle_mass_coord", ".robot.rh_rfmiddle_mass_coord",
                    ".robot.rh_lfmiddle_mass_coord",  ".robot.rh_thmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_mfproximal_mass_coord",
                    ".robot.rh_rfproximal_mass_coord", ".robot.rh_lfproximal_mass_coord", ".robot.rh_thproximal_mass_coord")
frames_names = ("fftip", "mftip", "rftip", "lftip", "thtip", "ffmiddle", "mfmiddle", "rfmiddle", "lfmiddle", "thmiddle", "ffprox", 
                "mfprox", "rfprox", "lfprox", "thprox")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, "box position") ; id = "$(frames_names[i]) box diff" )
    for j in 1:3
        add_coordinate!(vms, CoordSlice("$(frames_names[i]) box diff", SVector(j)); id = "$(frames_names[i]) box diff dimension $(j)")
        add_coordinate!(vms, CoordNorm("$(frames_names[i]) box diff dimension $(j)"); id = "$(frames_names[i]) box norm dimension $(j)")
        add_coordinate!(vms, CoordDifference("$(frames_names[i]) box norm dimension $(j)","box dimension $(j)"); id = "shifted $(frames_names[i]) box norm dimension $(j)")

        add_component!(vms, ReLUSpring(0.0, "shifted $(frames_names[i]) box norm dimension $(j)", true); id="$(frames_names[i]) dimension $(j) repulsive spring")
        add_component!(vms, RectifiedDamper(0.0, "$(frames_names[i]) box norm dimension $(j)", (0.0, 1.1*box_dimensions[j]), true, false); id="$(frames_names[i]) dimension $(j) damper")
    end
end

Hand motion

In [7]:
# TARGET REACHING
add_coordinate!(vms,  ConstCoord(box_position);  id="Target position")

add_coordinate!(vms, CoordDifference(".robot.rh_fftip_mass_coord", "Target position"); id="ff position error")
add_coordinate!(vms, CoordDifference(".robot.rh_mftip_mass_coord", "Target position"); id="mf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_rftip_mass_coord", "Target position"); id="rf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_lftip_mass_coord", "Target position"); id="lf position error")
add_coordinate!(vms, CoordDifference(".robot.rh_thtip_mass_coord", "Target position"); id="th position error")

K_matrix = SMatrix{3, 3}(10., 0., 0., 0., 10., 0., 0., 0., 10.)
D_matrix = SMatrix{3, 3}(10., 0., 0., 0., 10., 0., 0., 0., 10.)

add_component!(vms, LinearSpring(K_matrix, "ff position error"); id="ff spring")
add_component!(vms, LinearDamper(D_matrix, "ff position error"); id="ff damper")

add_component!(vms, LinearSpring(K_matrix, "mf position error"); id="mf spring")
add_component!(vms, LinearDamper(D_matrix, "mf position error"); id="mf damper")

add_component!(vms, LinearSpring(K_matrix, "rf position error"); id="rf spring")
add_component!(vms, LinearDamper(D_matrix, "rf position error"); id="rf damper")

add_component!(vms, LinearSpring(K_matrix, "lf position error"); id="lf spring")
add_component!(vms, LinearDamper(D_matrix, "lf position error"); id="lf damper")

add_component!(vms, LinearSpring(K_matrix, "th position error"); id="th spring")
add_component!(vms, LinearDamper(D_matrix, "th position error"); id="th damper")

"th damper"

## Simulating the Robot

### Setting Up the Simulation

In [8]:
function setup_box_collision_model(cache, repulsed_frames, frames_names)
    repulsed_frames_coord_ID = []
    repulsive_springs_damper_ID = []
    for i in 1:length(repulsed_frames)
        frame = repulsed_frames[i]
        push!(repulsed_frames_coord_ID, get_compiled_coordID(cache, frame))
        frame_springs_dampers_vec = []
        for j in 1:3
            push!(frame_springs_dampers_vec, get_compiled_componentID(cache, "$(frames_names[i]) dimension $(j) repulsive spring"))
            push!(frame_springs_dampers_vec, get_compiled_componentID(cache, "$(frames_names[i]) dimension $(j) damper"))
        end
        push!(repulsive_springs_damper_ID, frame_springs_dampers_vec)
    end

    return box_position, box_dimensions, repulsed_frames_coord_ID, repulsive_springs_damper_ID
end

function f_setup(cache)

    box_collision_args = setup_box_collision_model(cache, repulsed_frames, frames_names)

    return get_compiled_coordID(cache, "Target position"), box_collision_args
    
end

function update_box_collision_model(cache, collision_args)
    box_position, box_dimensions, repulsed_frames_coord_ID, repulsive_springs_damper_ID = collision_args
    margin = 0.001
    for i in 1:length(repulsed_frames_coord_ID)
        frame_pos = configuration(cache, repulsed_frames_coord_ID[i])
        for j in 1:3
            # get the indices different from j
            others = filter(x -> x ≠ j, 1:3) 
            #Check if the position of the frame is inside "the field of action" of the spring
            if abs(frame_pos[others[1]] - box_position[others[1]]) < (box_dimensions[others[1]]-margin) && abs(frame_pos[others[2]] - box_position[others[2]]) < (box_dimensions[others[2]]-margin)
                cache[repulsive_springs_damper_ID[i][2*j-1]] = remake(cache[repulsive_springs_damper_ID[i][2*j-1]] ; stiffness = 1000.0)
                cache[repulsive_springs_damper_ID[i][2*j]] = remake(cache[repulsive_springs_damper_ID[i][2*j]] ; damping = 100.0)          
            else
                cache[repulsive_springs_damper_ID[i][2*j-1]] = remake(cache[repulsive_springs_damper_ID[i][2*j-1]] ; stiffness = 0.0)
                cache[repulsive_springs_damper_ID[i][2*j]] = remake(cache[repulsive_springs_damper_ID[i][2*j]] ; damping = 0.0)
            end
        end
    end
end

function f_control(cache, t, args, extra)
    target_pos_id, collision_args = args 
    update_box_collision_model(cache, collision_args)

    if t > 3 && t < 6
        cache[target_pos_id] = remake(cache[target_pos_id] ; coord_data = ConstCoord(SVector(0.05, -0.065, 0.32)))
    end

    if t > 6
        cache[target_pos_id] = remake(cache[target_pos_id] ; coord_data = ConstCoord(SVector(0.00, -0.065, 0.32)))
    end
end

tspan = (0., 10.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with repulsive box"

using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-3, reltol=1e-3);

[ Info: Simulating shadow robot with repulsive box
ODE   0%|█                                              |  ETA: N/A
┌ Warning: Automatic dt set the starting dt as NaN, causing instability.
│ Exiting.
└ @ OrdinaryDiffEq C:\Users\bilou\.julia\packages\OrdinaryDiffEq\NBaQM\src\solve.jl:578
┌ Warning: NaN dt detected. Likely a NaN value in the state, parameters, or
│ derivative value caused this outcome.
└ @ SciMLBase C:\Users\bilou\.julia\packages\SciMLBase\2HZ5m\src\integrator_interface.jl:570
ODE 100%|███████████████████████████████████████████████| Time: 0:00:02


### Visualizing the Results

In [11]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))


robotvisualize!(ls, plotting_kcache)

box_visual_dimensions = box_dimensions - 0.004*SVector(1.,1.,1.)
box = Rect3f(Point3f(box_position) - Vec3f(box_visual_dimensions), 2*Vec3f(box_visual_dimensions))
mesh!(ls, box; color=:magenta, transparency=true)


savepath = joinpath(module_path, "docs/src/assets/shadowhand_box_collision_model.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);